# Routes

> FastHTML route handlers for settings interface

In [ ]:
#| default_exp routes

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from pathlib import Path
from typing import Dict, Any, Optional, Callable
from fasthtml.common import *

from cjm_fasthtml_settings.components.alerts import create_error_alert, create_success_alert
from cjm_fasthtml_settings.core.html_ids import HtmlIds
from cjm_fasthtml_settings.core.config import DEFAULT_CONFIG_DIR
from cjm_fasthtml_settings.core.schemas import registry
from cjm_fasthtml_settings.core.utils import (
    save_config,
    get_default_values_from_schema,
    convert_form_data_to_config,
)
from cjm_fasthtml_settings.components.dashboard import settings_content, render_schema_settings_content
from cjm_fasthtml_settings.components.forms import create_settings_form_container
from cjm_fasthtml_settings.components.sidebar import create_oob_sidebar_menu, create_sidebar_menu

## Configuration

Module-level configuration that can be customized by users before importing the router.

In [ ]:
#| export
# Configuration settings that users can override
class RoutesConfig:
    """Configuration for settings routes behavior.
    
    Users can modify these before importing the router:
    
    Example:
        ```python
        from cjm_fasthtml_settings.routes import config
        config.config_dir = Path("my_configs")
        config.default_schema = "database"
        config.wrap_with_layout = my_layout_function
        config.plugin_registry = my_plugin_registry
        
        # Now import the router
        from cjm_fasthtml_settings.routes import settings_ar
        ```
    """
    config_dir: Path = DEFAULT_CONFIG_DIR
    default_schema: str = "general"
    menu_section_title: str = "Settings"
    wrap_with_layout: Optional[Callable] = None  # Optional function to wrap full page content
    plugin_registry: Optional[Any] = None  # Optional plugin registry (must implement PluginRegistryProtocol)

# Module-level config instance
config = RoutesConfig()

## Helper Functions

In [ ]:
#| export
def _resolve_schema(id: str):
    """Resolve schema from ID using the registry.
    
    Handles both direct schemas and grouped schemas via registry.resolve_schema().
    
    Returns:
        tuple: (schema, error_message) - schema is None if error occurred
    """
    return registry.resolve_schema(id)

In [ ]:
#| export
def _handle_htmx_request(request, content_fn: Callable, *args, **kwargs):
    """Handle HTMX vs full page response pattern."""
    content = content_fn(*args, **kwargs)
    
    # Check if this is an HTMX request
    if request.headers.get('HX-Request'):
        return content
    
    # For full page requests, wrap with page layout if provided
    if config.wrap_with_layout:
        return config.wrap_with_layout(content)
    return content

In [ ]:
#| export
def _create_settings_response(
    schema: Dict[str, Any],
    values: Dict[str, Any],
    save_url: str,
    reset_url: str,
    alert_msg,
    sidebar_id: str
):
    """Create standardized settings form response with sidebar."""
    return Div(
        create_settings_form_container(
            schema=schema,
            values=values,
            post_url=save_url,
            reset_url=reset_url,
            alert_message=alert_msg
        ),
        create_oob_sidebar_menu(
            schemas=registry.get_all(),
            active_schema=sidebar_id,
            config_dir=config.config_dir,
            menu_section_title=config.menu_section_title,
            plugin_registry=config.plugin_registry
        )
    )

## API Router and Routes

The module-level `settings_ar` router can be directly imported and attached to your FastHTML app.

In [ ]:
#| export
# Module-level API router
settings_ar = APIRouter(prefix="/settings")

In [ ]:
#| export
@settings_ar
def index(request, id: str = None):
    """Main settings page.
    
    Args:
        request: FastHTML request object
        id: Schema ID to display (defaults to config.default_schema)
    """
    if id is None:
        id = config.default_schema
        
    schema, _ = _resolve_schema(id)
    
    return _handle_htmx_request(
        request,
        settings_content,
        request,
        schema,
        registry.get_all(),
        config.config_dir,
        menu_section_title=config.menu_section_title,
        plugin_registry=config.plugin_registry
    )

In [ ]:
#| export
@settings_ar
def load_form(id: str = None):
    """Async endpoint that loads the settings form.
    
    Args:
        id: Schema ID to load (defaults to config.default_schema)
    """
    from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex
    from cjm_fasthtml_tailwind.utilities.sizing import min_h
    from cjm_fasthtml_tailwind.core.base import combine_classes
    
    if id is None:
        id = config.default_schema
        
    schema, _ = _resolve_schema(id)
    
    return Div(
        render_schema_settings_content(
            schema,
            config.config_dir
        ),
        id=HtmlIds.SETTINGS_CONTENT,
        cls=combine_classes(flex(1), min_h(0))
    )

In [ ]:
#| export
@settings_ar
async def save(request, id: str):
    """Save configuration handler.
    
    Args:
        request: FastHTML request object
        id: Schema ID to save
    """
    schema, error_msg = _resolve_schema(id)
    if error_msg:
        return create_error_alert(error_msg)
    
    form_data = await request.form()
    config_data = convert_form_data_to_config(form_data, schema)
    
    # Save configuration
    if save_config(id, config_data, config.config_dir):
        return _create_settings_response(
            schema=schema,
            values=config_data,
            save_url=save.to(id=id),
            reset_url=reset.to(id=id),
            alert_msg=create_success_alert(f"Configuration saved for {schema.get('title')}"),
            sidebar_id=id
        )
    else:
        return create_error_alert(f"Failed to save {schema.get('title')} configuration")

In [ ]:
#| export
@settings_ar
def reset(id: str):
    """Reset configuration to defaults handler.
    
    Args:
        id: Schema ID to reset
    """
    schema, error_msg = _resolve_schema(id)
    if error_msg:
        return create_error_alert(error_msg)
    
    # Use only default values
    values = get_default_values_from_schema(schema)
    
    return _create_settings_response(
        schema=schema,
        values=values,
        save_url=save.to(id=id),
        reset_url=reset.to(id=id),
        alert_msg=create_success_alert("Configuration reset to defaults"),
        sidebar_id=id
    )

## Plugin Routes (Optional)

These routes are only active if `config.plugin_registry` is set.

In [ ]:
#| export
@settings_ar
def plugin_reset(id: str):
    """Reset plugin configuration to defaults handler.
    
    Args:
        id: Plugin unique ID
    """
    if not config.plugin_registry:
        return create_error_alert("Plugin system not configured")
    
    plugin_metadata = config.plugin_registry.get_plugin(id)
    if not plugin_metadata:
        return create_error_alert("Plugin not found")
    
    schema = plugin_metadata.config_schema
    values = get_default_values_from_schema(schema)
    
    return _create_settings_response(
        schema=schema,
        values=values,
        save_url=plugin_save.to(id=id),
        reset_url=plugin_reset.to(id=id),
        alert_msg=create_success_alert("Configuration reset to defaults"),
        sidebar_id=id
    )

In [ ]:
#| export
@settings_ar
async def plugin_save(request, id: str):
    """Save plugin configuration handler.
    
    Args:
        request: FastHTML request object
        id: Plugin unique ID
    """
    if not config.plugin_registry:
        return create_error_alert("Plugin system not configured")
    
    plugin_metadata = config.plugin_registry.get_plugin(id)
    if not plugin_metadata:
        return create_error_alert("Plugin not found")
    
    form_data = await request.form()
    schema = plugin_metadata.config_schema
    config_data = convert_form_data_to_config(form_data, schema)
    
    # Save configuration
    if config.plugin_registry.save_plugin_config(id, config_data):
        return _create_settings_response(
            schema=schema,
            values=config_data,
            save_url=plugin_save.to(id=id),
            reset_url=plugin_reset.to(id=id),
            alert_msg=create_success_alert(f"Configuration saved for {plugin_metadata.title}"),
            sidebar_id=id
        )
    else:
        return create_error_alert("Failed to save configuration")

In [ ]:
#| export
@settings_ar
def plugin(request, id: str):
    """Plugin settings page.
    
    Args:
        request: FastHTML request object
        id: Plugin unique ID
    """
    if not config.plugin_registry:
        return create_error_alert("Plugin system not configured")
    
    plugin_metadata = config.plugin_registry.get_plugin(id)
    if not plugin_metadata:
        # Fallback to general settings if plugin not found
        schema, _ = _resolve_schema(config.default_schema)
        return _handle_htmx_request(
            request,
            settings_content,
            request,
            schema,
            registry.get_all(),
            config.config_dir,
            menu_section_title=config.menu_section_title
        )
    
    # Load plugin config
    saved_config = config.plugin_registry.load_plugin_config(id)
    schema = plugin_metadata.config_schema
    default_values = get_default_values_from_schema(schema)
    values = {**default_values, **saved_config}
    
    # Check if HTMX request for settings-content only
    if request.headers.get('HX-Request') and request.headers.get('HX-Target') == HtmlIds.SETTINGS_CONTENT:
        # Return content with updated sidebar
        return Div(
            create_settings_form_container(
                schema=schema,
                values=values,
                post_url=plugin_save.to(id=id),
                reset_url=plugin_reset.to(id=id),
                use_alert_container=True
            ),
            create_oob_sidebar_menu(
                schemas=registry.get_all(),
                active_schema=id,
                config_dir=config.config_dir,
                menu_section_title=config.menu_section_title,
                plugin_registry=config.plugin_registry
            )
        )
    
    # Full page layout
    from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, gap, flex
    from cjm_fasthtml_tailwind.utilities.spacing import p, m
    from cjm_fasthtml_tailwind.utilities.sizing import container, max_w, w, min_h
    from cjm_fasthtml_tailwind.core.base import combine_classes
    
    content = Div(
        # Sidebar
        Div(
            create_sidebar_menu(
                schemas=registry.get_all(),
                active_schema=id,
                config_dir=config.config_dir,
                include_wrapper=True,
                menu_section_title=config.menu_section_title
            ),
            cls=combine_classes(w(64), flex.shrink_0)
        ),
        # Main content
        Div(
            create_settings_form_container(
                schema=schema,
                values=values,
                post_url=plugin_save.to(id=id),
                reset_url=plugin_reset.to(id=id),
                use_alert_container=True
            ),
            id=HtmlIds.SETTINGS_CONTENT,
            cls=combine_classes(flex(1), min_h(0))
        ),
        cls=combine_classes(
            flex_display,
            gap(6),
            container,
            max_w._7xl,
            m.x.auto,
            p(6)
        )
    )
    
    return _handle_htmx_request(request, lambda: content)

## Usage Example

In [ ]:
# Example: Using the settings router in your app
from cjm_fasthtml_settings.core.schemas import registry
from cjm_fasthtml_settings.core.config import get_app_config_schema

# First, register your schemas
registry.register(get_app_config_schema(app_title="My App", include_theme=False))
registry.register({
    "name": "database",
    "title": "Database Settings",
    "type": "object",
    "properties": {
        "host": {"type": "string", "title": "Host", "default": "localhost"}
    }
})

# Optional: Configure routes behavior
from cjm_fasthtml_settings.routes import config
from pathlib import Path
config.config_dir = Path("my_configs")
config.default_schema = "general"

# Then import and use the router
from cjm_fasthtml_settings.routes import settings_ar

print(f"Settings router prefix: {settings_ar.prefix}")
print(f"Registered schemas: {registry.list_schemas()}")
print(f"Config directory: {config.config_dir}")

Settings router prefix: /settings
Registered schemas: ['general', 'database']
Config directory: my_configs


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()